In [1]:
# 导入oled屏幕库 Import oled screen library
import sys
sys.path.append('/home/pi/software/oled_yahboom/')
from yahboom_oled import *
# 创建oled对象 Create an oled object
oled = Yahboom_OLED(debug=False)
# 导入组件 Importing Components
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=640, height=640)  #设置摄像头显示组件 Set up the camera display component
display(image_widget) 

Image(value=b'', format='jpeg', height='640', width='640')

In [2]:
# 导入cv相关库 Import cv related libraries
import cv2
import numpy as np
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
# 导入依赖包 Import dependency packages
import hyperlpr3 as lpr3
 
# 将BGR图像转换为JPEG格式的字节流 Convert a BGR image to a JPEG byte stream
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 在图像上绘制车牌框及文字 Draw the license plate frame and text on the image
def draw_plate_on_image(img, box, text, font):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (225, 32, 39), 2, cv2.LINE_AA)
    cv2.rectangle(img, (x1, y1 - 20), (x2, y1), (225, 32, 39), -1)
    data = Image.fromarray(img)
    draw = ImageDraw.Draw(data)
    draw.text((x1 + 1, y1 - 18), text, (255, 255, 255), font=font)
    res = np.asarray(data)
    return res
 
 
# 中文字体加载 Chinese font loading
font_ch = ImageFont.truetype("platech.ttf", 20, 0)
 
# 实例化识别对象 Instantiate the recognition object
catcher = lpr3.LicensePlateCatcher(detect_level=lpr3.DETECT_LEVEL_LOW)#DETECT_LEVEL_HIGH640*640
# 读取图片 Reading pictures
image = cv2.imread("沪AE97033.png")
try:
    oled.init_oled_process() #初始化oled进程 Initialize oled process
    # 执行识别算法 Execute the recognition algorithm
    results = catcher(image)
    for code, confidence, type_idx, box in results:
        # 解析数据并绘制 Parsing and plotting data
        text = f"{code} - {confidence:.2f}"
        image = draw_plate_on_image(image, box, text, font=font_ch)
    oled.clear()
    carcher_str=f'carcher : {code}'
    confidence_str=f'confidence:{confidence:.2f}'
    oled.add_cnline(carcher_str, 1)
    oled.add_line(confidence_str, 3)
    oled.refresh()
    image_widget.value = bgr8_to_jpeg(image)
    print(text)
    # 显示检测结果 Display test results
    # cv2.imshow("LicensePlate", image)
    # cv2.waitKey(0)
except KeyboardInterrupt:
    cv2.destroyAllWindows()

Process 1297 has been terminated.
沪AE97033 - 0.99


In [3]:
# 结束 Finish
# 恢复屏幕基础数据显示 Restore basic data display on screen
os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")

0